In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import pandas as pd
import numpy as np
from pathlib import Path
from math import floor

# Get all riders in RacerX Motocross Riders List

In [2]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [3]:
all_riders = []

In [4]:
url = 'https://racerxonline.com/mx/riders/a'
    
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(url,headers=hdr)
page = urlopen(req)

soup = BeautifulSoup(page)

riders = soup.find_all("a", href=re.compile("/rider/"))
test = soup.find_all('h3', class_='link_title')

In [5]:
for letter in alphabet:
    # print(letter)
    url = 'https://racerxonline.com/mx/riders/{}'.format(letter)
    
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    
    soup = BeautifulSoup(page)
    
    riders = soup.find_all(class_='link_title')
        
    riders_list = [rider.text for rider in riders]
    
    all_riders = all_riders + riders_list

In [6]:
all_riders[-1]

' Darryl Zumstein '

In [7]:
all_riders = [r.strip() for r in all_riders]

In [8]:
all_riders[-1]

'Darryl Zumstein'

In [9]:
all_riders = [r.lower() for r in all_riders]

# Get these riders' Loretta's Results

In [10]:
test = 'https://llvault.racerxonline.com/rider/destry-abbott/races'

hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(test,headers=hdr)
page = urlopen(req)

soup = BeautifulSoup(page)

In [11]:
results = soup.find_all('table')

In [12]:
len(results)

1

In [13]:
race = results[0].find_all('tr', class_='ll')
len(race)

4

In [18]:
info = race.find_all('td')

In [19]:
position = info[0].text.strip()
race_class = info[1].find('a').text.strip()
date = info[2].text.strip()
bike = info[3].text.strip()

In [20]:
race_class

'Vet 30 Plus'

In [21]:
all_riders[0]

'destry abbott'

In [14]:
df_rider = []
df_position = []
df_class = []
df_date = []
df_bike = []

In [15]:
no_results = []

In [16]:
for rider in all_riders:
    name_formatted = rider.replace(' ', '-')
    
    ll_url = 'https://llvault.racerxonline.com/rider/{}/races'.format(name_formatted)
    
    try:
        hdr = {'User-Agent': 'Mozilla/5.0'}
        req = Request(ll_url,headers=hdr)
        page = urlopen(req)
    except:
        no_results.append(rider)
        continue

    soup = BeautifulSoup(page)
    
    results = soup.find_all('table')
    
    race = results[0].find_all('tr', class_='ll')
    
    for i in range(len(race)):
        race2 = race[i]
        info = race2.find_all('td')

        position = info[0].text.strip()
        race_class = info[1].find('a').text.strip()
        date = info[2].text.strip()
        bike = info[3].text.strip()

        df_rider.append(rider)
        df_position.append(position)
        df_class.append(race_class)
        df_date.append(date)
        df_bike.append(bike)

In [17]:
df = pd.DataFrame()

In [18]:
df['rider'] = df_rider
df['overall_result'] = df_position
df['race_class'] = df_class
df['date'] = df_date
df['bike'] = df_bike

In [19]:
df.head()

,rider,overall_result,race_class,date,bike
0,destry abbott,30,Plus 25,2005,Kawasaki
1,destry abbott,4,Vet 30 Plus,2005,Kawasaki
2,destry abbott,5,Vet 35+,2012,Kawasaki
3,destry abbott,3,Senior 40+,2012,Kawasaki
4,bryan abernathy,36,125 A Modified - Youth,1982,Kawasaki


In [20]:
df.to_csv("C:/research_data/lorettas_results.csv")

In [21]:
len(df)

9290

# Get RacerX Data For Every Rider Available

In [2]:
df = pd.read_csv("C:/research_data/lorettas_results.csv")

In [3]:
unique_riders = list(df['rider'].unique())

In [4]:
no_racerx_results = []

In [5]:
url = 'https://racerxonline.com/rider/eli-tomac/races'
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(url,headers=hdr)
page = urlopen(req)

soup = BeautifulSoup(page)

In [6]:
race_results = soup.find_all('table', class_='ui_table zebra mod_mobile')

In [7]:
# races are in <tr> tags
tomac_races = race_results[0].find_all('tr')
tomac_races = tomac_races[1:]
example = tomac_races[0]
example_results = example.find_all('td')
example_results[1].find_all('a')[0].text

' Denver '

In [8]:
example_results

[<td>22</td>,
 <td><div class="tag">Supercross </div><a class="block" href="/sx/2023/denver/450sx"> Denver </a></td>,
 <td>450SX</td>,
 <td>May 6, 2023</td>,
 <td> <a class="block" href="/brand/yamaha/2023">Yamaha YZ450F</a> </td>]

In [5]:
riders = []
positions = []
races_ = []
race_classes = []
dates = []
bikes = []

In [6]:
rider_count = 0

In [7]:
for rider in unique_riders:
    rider_count += 1
    
    if floor(rider_count/len(unique_riders)*100) % 10 == 0 & floor(rider_count/len(unique_riders)*100) % 10 != 0:
        print(floor(rider_count/len(unique_riders)))
        
    url_name = rider.replace(' ', '-')
    url = 'https://racerxonline.com/rider/{}/races'.format(url_name)
    
    try:
        hdr = {'User-Agent': 'Mozilla/5.0'}
        req = Request(url,headers=hdr)
        page = urlopen(req)
    except:
        no_racerx_results.append(rider)
        continue
        
    soup = BeautifulSoup(page)
    
    race_results = soup.find_all('table', class_='ui_table zebra mod_mobile')
    
    races = race_results[0].find_all('tr')
    races = races[1:]
    
    for i in range(len(races)):
        example = races[i]
        example_results = example.find_all('td')
        
        finish_pos = example_results[0].text.strip()
        race = example_results[1].find_all('a')[0].text.strip()
        race_class = example_results[2].text.strip()
        race_date = example_results[3].text
        bike = example_results[4].find_all('a')[0].text.strip()
        
        riders.append(rider)
        positions.append(finish_pos)
        races_.append(race)
        race_classes.append(race_class)
        dates.append(race_date)
        bikes.append(bike)

In [8]:
racerx_df = pd.DataFrame()

In [9]:
racerx_df['rider'] = riders
racerx_df['position'] = positions
racerx_df['race'] = races_
racerx_df['race_class'] = race_classes
racerx_df['date'] = dates
racerx_df['bike'] = bikes

In [10]:
racerx_df.head()

,rider,position,race,race_class,date,bike
0,destry abbott,40,Thunder Valley Motocross,MX,"July 22, 2007",Kawasaki KX450F
1,bryan abernathy,19,Houston,125E/WSX,"April 20, 1985",Honda
2,bryan abernathy,22,Washougal Motocross,125MX,"August 19, 1984",Honda
3,bryan abernathy,13,Broome-Tioga Sports Center,125MX,"July 22, 1984",Honda
4,bryan abernathy,18,RedBud Track 'n Trail,125MX,"July 1, 1984",Honda


In [11]:
racerx_df.to_csv("C:/research_data/racerx_results.csv")